In [2]:
# libs
import os
import pandas as pd
import requests
from dotenv import load_dotenv #pip3 install python-dotenv
# pip3 install lxml
import openpyxl
#### sudz4 -> future feature -> try to use xlwings to write to excel

# load secrets
load_dotenv()

True

In [3]:
def get_historical_data(symbol, lookback_days=180):
    iex_key = os.getenv("IEX_API_KEY") 
    historical_url = f'https://cloud.iexapis.com/v1/stock/{symbol}/chart/{lookback_days}d?token={iex_key}'

    historical_data = requests.get(historical_url).json()
    df = pd.DataFrame(historical_data)
    return df[['date', 'high', 'low', 'close']]

# Fetch historical data
ticker = 'PLTR' # Palantir Technologies Inc
#### sudz4 -> think about getting a list, can do list of SP500 or sector, pass the list to the function and get the data
historical_df = get_historical_data(ticker)

# Display the historical data
# print(historical_df)

# Convert the 'date' column to datetime objects
historical_df['date'] = pd.to_datetime(historical_df['date'])

# Sort the DataFrame by the 'date' column
historical_df = historical_df.sort_values(by='date')

In [4]:
# display historical data info
print()
print('Periods:', len(historical_df))
print('Period Start Date:', historical_df['date'].iloc[0])
print('Period End Date:', historical_df['date'].iloc[-1])
print()

# display historical data partial view
print('Historical Data START')
print(historical_df.head(5))
print()
print('Historical Data END')
print(historical_df.tail(5)) # last 5 rows


Periods: 124
Period Start Date: 2023-08-07 00:00:00
Period End Date: 2024-02-01 00:00:00

Historical Data START
        date   high    low  close
0 2023-08-07  18.39  17.01  17.99
1 2023-08-08  17.49  16.10  17.04
2 2023-08-09  16.92  15.10  15.25
3 2023-08-10  15.87  15.25  15.41
4 2023-08-11  15.58  14.91  15.41

Historical Data END
          date     high    low  close
119 2024-01-26  16.7500  16.34  16.35
120 2024-01-29  17.1100  16.43  17.09
121 2024-01-30  17.2400  16.66  16.74
122 2024-01-31  16.7600  16.07  16.09
123 2024-02-01  16.4868  16.03  16.33


In [ ]:
# make pandas dataframe setting show all rows and columns
pd.set_option('display.max_rows', None)

#display historical data FULL chart view
# historical_df # uncomment to display full chart view

In [5]:
# Calculate Ichimoku components
high_9 = historical_df['high'].rolling(window=9).max()
low_9 = historical_df['low'].rolling(window=9).min()
historical_df['tenkan_sen'] = (high_9 + low_9) / 2

high_26 = historical_df['high'].rolling(window=26).max()
low_26 = historical_df['low'].rolling(window=26).min()
historical_df['kijun_sen'] = (high_26 + low_26) / 2

historical_df['senkou_span_a'] = ((historical_df['tenkan_sen'] + historical_df['kijun_sen']) / 2).shift(26)

high_52 = historical_df['high'].rolling(window=52).max()
low_52 = historical_df['low'].rolling(window=52).min()
historical_df['senkou_span_b'] = ((high_52 + low_52) / 2).shift(26)

historical_df['chikou_span'] = historical_df['close'].shift(-26)

# print updated df
historical_df

,date,high,low,close,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
0,2023-08-07,18.3900,17.01,17.99,NaN,NaN,NaN,NaN,15.60
1,2023-08-08,17.4900,16.10,17.04,NaN,NaN,NaN,NaN,15.83
2,2023-08-09,16.9200,15.10,15.25,NaN,NaN,NaN,NaN,15.33
3,2023-08-10,15.8700,15.25,15.41,NaN,NaN,NaN,NaN,15.46
4,2023-08-11,15.5800,14.91,15.41,NaN,NaN,NaN,NaN,15.15
...,...,...,...,...,...,...,...,...,...
119,2024-01-26,16.7500,16.34,16.35,17.20015,17.00715,18.7750,18.165,NaN
120,2024-01-29,17.1100,16.43,17.09,17.20015,17.00715,18.7825,18.165,NaN
121,2024-01-30,17.2400,16.66,16.74,17.22515,17.00715,18.7850,18.165,NaN
122,2024-01-31,16.7600,16.07,16.09,17.21015,17.00715,18.8100,18.165,NaN


In [10]:
# save dataframe to .xlsx file
# header is the row with the column names
pd.DataFrame(historical_df).to_excel('/Users/sudz4/Desktop/FINANCIAL-SYSTEMS/dark-trader/ichimoku_cloud/ichimoku_data/PLTR_180d_ichimoku.xlsx', header=True, index=False)

# print success message
print('PLTR_180d_ichimoku.xlsx file saved successfully!')

PLTR_180d_ichimoku.xlsx file saved successfully!
